In [16]:
import os
import glob
import pandas as pd

# all datafiles for each month from May 2023 to April 2025 are downloaded and stored in the dataset folder 
folder_path = "C:/Users/rohit/OneDrive/Desktop/Uni/Dissertation/dataset"
output_path = "C:/Users/rohit/OneDrive/Desktop/Uni/Dissertation/cleaned_dataset"
# combining all csv files
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Sort files to ensure order (May 2023 → April 2025)
all_files.sort()

# Read and combine all files
df_list = []
for i, file in enumerate(all_files):
    if i == 0:
        # Read with header for the first file
        df = pd.read_csv(file)
    else:
        # Skip header row for other files
        df = pd.read_csv(file, header=0)
    df_list.append(df)

# Concatenate everything
final_df = pd.concat(df_list, ignore_index=True)

# Save to a single CSV
output_file = os.path.join(output_path, "combined_city_of_london_crime.csv")
final_df.to_csv(output_file, index=False)

print(f"Combined CSV saved as: {output_file}")


Combined CSV saved as: C:/Users/rohit/OneDrive/Desktop/Uni/Dissertation/cleaned_dataset\combined_city_of_london_crime.csv


In [17]:
# data quality checks and processing for temporal analysis
import pandas as pd
import uuid
from pathlib import Path

# Load & standardise schema
path = Path("C:/Users/rohit/OneDrive/Desktop/Uni/Dissertation/cleaned_dataset/combined_city_of_london_crime.csv")
df = pd.read_csv(path)
print("Initial row count after loading CSV:", len(df))

df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("/", "_", regex=False)
)

# Removing duplicate records based on unique identifier 'crime_id'
# also making sure random uuids are used to assign values to the blank 'crime_id' fields
if "crime_id" in df.columns:
    # Replace missing/blank crime_id with random UUIDs so we don't drop those records
    null_id_mask = df["crime_id"].isna() | (df["crime_id"].astype(str).str.strip() == "")
    df.loc[null_id_mask, "crime_id"] = [str(uuid.uuid4()) for _ in range(null_id_mask.sum())]

    # Drop duplicate rows having the same crime_id, keeping first occurrence
    df = df.drop_duplicates(subset=["crime_id"], keep="first")

print("Final row count after UUID assignment & duplicate removal:", len(df))

# Checking and Parsing the month field wherever needed
parsed = False

if "month" in df.columns:
    # UK Police format is typically YYYY-MM
    try:
        df["month_dt"] = pd.to_datetime(df["month"], format="%Y-%m")
    except Exception:
        df["month_dt"] = pd.to_datetime(df["month"], errors="coerce")
    parsed = True

if not parsed and "date" in df.columns:
    df["month_dt"] = pd.to_datetime(df["date"], errors="coerce")
    parsed = True

if not parsed and "reported_month" in df.columns:
    df["month_dt"] = pd.to_datetime(df["reported_month"], errors="coerce")
    parsed = True

if not parsed:
    # Fallback: look for any column with YYYY-MM strings
    inferred = None
    for c in df.columns:
        if df[c].dtype == object:
            s = df[c].astype(str)
            if s.str.match(r"^\d{4}-\d{2}$", na=False).any():
                inferred = c
                break
    if inferred is None:
        raise ValueError("No usable month/date field found (e.g., 'Month' in YYYY-MM).")
    df["month_dt"] = pd.to_datetime(df[inferred], format="%Y-%m", errors="coerce")

# Keep valid months only
df = df[~df["month_dt"].isna()].copy()

# creating tidy and readable time fields
df["year"] = df["month_dt"].dt.year
df["month_num"] = df["month_dt"].dt.month
df["month_name"] = df["month_dt"].dt.month_name()

# Creating new fields 'year_month_period' and 'year_month_label'
df["year_month_period"] = df["month_dt"].dt.to_period("M")          # e.g., 2024-05
df["year_month_label"] = df["month_dt"].dt.strftime("%B %Y")        # e.g., "May 2024"

# Save cleaned DataFrame to CSV
output_path = Path("C:/Users/rohit/OneDrive/Desktop/Uni/Dissertation/cleaned_dataset/cleaned_city_of_london_crime_for_temporal_analysis.csv")
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved as: {output_path.name}")


Initial row count after loading CSV: 19172
Final row count after UUID assignment & duplicate removal: 18777
Cleaned dataset saved as: cleaned_city_of_london_crime_for_temporal_analysis.csv
